In [13]:
#Import requied libraries
import websocket
import json
import time 
from kafka import KafkaProducer
from datetime import datetime

In [6]:
# Kafka config
producer = KafkaProducer(
    bootstrap_servers='35.154.244.134:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)
topic = 'crypto_ticker_stream'

In [12]:
# Handle messages from Binance WebSocket
def on_message(ws, message):
    parsed = json.loads(message)
    data = parsed['data']

    result = {
        "symbol": data['s'],
        "price": float(data['c']),
        "price_change": float(data['p']),
        "percent_change": float(data['P']),
        "volume": float(data['v']),
        "time": datetime.fromtimestamp(data['E'] / 1000).strftime('%Y-%m-%d %H:%M:%S')
    }

    # Send to Kafka
    producer.send(topic, value=result)
    print(f"Sent to Kafka: {result}")

    # Sleep for 3 seconds
    time.sleep(3)

def on_error(ws, error):
    print("Error:", error)

def on_close(ws, close_status_code, close_msg):
    print("WebSocket closed")

def on_open(ws):
    print("WebSocket connection opened")

# Binance WebSocket URL for multiple symbols
symbols = ["btcusdt", "ethusdt", "bnbusdt"]
streams = "/".join(f"{sym}@ticker" for sym in symbols)
url = f"wss://stream.binance.com:9443/stream?streams={streams}"

# Start WebSocket
ws = websocket.WebSocketApp(
    url,
    on_open=on_open,
    on_message=on_message,
    on_error=on_error,
    on_close=on_close
)

ws.run_forever()


WebSocket connection opened
Sent to Kafka: {'symbol': 'BTCUSDT', 'price': 105453.4, 'price_change': 509.01, 'percent_change': 0.485, 'volume': 7355.34369, 'time': '2025-06-08 12:59:36'}
Sent to Kafka: {'symbol': 'ETHUSDT', 'price': 2514.01, 'price_change': 21.91, 'percent_change': 0.879, 'volume': 224374.5985, 'time': '2025-06-08 12:59:36'}
Sent to Kafka: {'symbol': 'BNBUSDT', 'price': 649.74, 'price_change': 0.56, 'percent_change': 0.086, 'volume': 77412.383, 'time': '2025-06-08 12:59:36'}
Sent to Kafka: {'symbol': 'BTCUSDT', 'price': 105453.4, 'price_change': 509.01, 'percent_change': 0.485, 'volume': 7355.34327, 'time': '2025-06-08 12:59:37'}
Sent to Kafka: {'symbol': 'ETHUSDT', 'price': 2514.01, 'price_change': 21.91, 'percent_change': 0.879, 'volume': 224374.7146, 'time': '2025-06-08 12:59:37'}
Sent to Kafka: {'symbol': 'BNBUSDT', 'price': 649.74, 'price_change': 0.56, 'percent_change': 0.086, 'volume': 77412.405, 'time': '2025-06-08 12:59:37'}
Sent to Kafka: {'symbol': 'BTCUSDT',

False

In [11]:
producer.flush()